<a href="https://colab.research.google.com/github/Zahra-Mhdi/Deep-Learning-Exercises/blob/main/Session_4_CNN_Exercise_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 Exercise 2:

1. Try increasing num_epochs to 5 or 10 .

2. Replace ReLU with Sigmoid or Tanh .

3. Change kernel size to 5x5 .

4. Add a dropout layer after fc1.

5. Replace Adam with SGD (different learning rate).

6. Train a deeper CNN on CIFAR-10 using datasets.CIFAR10.

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

x = torch.tensor([[1.,2.,3.],
                  [4.,5.,6.],
                  [7.,8.,9.]]).unsqueeze(0).unsqueeze(0)
kernel = torch.tensor([[1.,0.],
                       [0.,-1.]]).unsqueeze(0).unsqueeze(0)
output = F.conv2d(x, kernel, stride=1, padding=0)
print("Input:", x)
print("kernel:", kernel)
print("output:", output)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 5, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, 5, padding=2)
        self.fc1 = nn.Linear(16*7*7, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*7*7)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = SimpleCNN().to(device)
print(model)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

num_epochs = 10
for epoch in range(num_epochs):
    total_loss = 0
    correct = 0
    total = 0
    model.train()
    for image, label in train_loader:
        image, label = image.to(device), label.to(device)
        optimizer.zero_grad()
        output = model(image)
        loss = loss_fn(output, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()
    train_acc = 100 * correct / total
    print(f"MNIST Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%")

correct = 0
total = 0
with torch.no_grad():
    model.eval()
    for image, label in test_loader:
        image, label = image.to(device), label.to(device)
        output = model(image)
        _, predicted = torch.max(output.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()
print(f"MNIST Test Accuracy: {100 * correct / total:.2f}%")

transform_cifar = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])

train_dataset_cifar = datasets.CIFAR10(root='./data_cifar', train=True, download=True, transform=transform_cifar)
test_dataset_cifar = datasets.CIFAR10(root='./data_cifar', train=False, download=True, transform=transform_cifar)

train_loader_cifar = DataLoader(train_dataset_cifar, batch_size=64, shuffle=True)
test_loader_cifar = DataLoader(test_dataset_cifar, batch_size=64, shuffle=False)

class DeeperCIFARCNN(nn.Module):
    def __init__(self):
        super(DeeperCIFARCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 128, 3, padding=1)
        self.fc1 = nn.Linear(128 * 8 * 8, 256)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = self.pool(F.relu(self.conv4(x)))
        x = x.view(-1, 128 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model_cifar = DeeperCIFARCNN().to(device)
print(model_cifar)

loss_fn_cifar = nn.CrossEntropyLoss()
optimizer_cifar = torch.optim.Adam(model_cifar.parameters(), lr=0.001)

num_epochs_cifar = 10
for epoch in range(num_epochs_cifar):
    model_cifar.train()
    total_loss = 0
    correct = 0
    total = 0
    for images, labels in train_loader_cifar:
        images, labels = images.to(device), labels.to(device)
        optimizer_cifar.zero_grad()
        outputs = model_cifar(images)
        loss = loss_fn_cifar(outputs, labels)
        loss.backward()
        optimizer_cifar.step()
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    train_acc = 100 * correct / total
    print(f"CIFAR10 Epoch {epoch+1}/{num_epochs_cifar}, Loss: {total_loss/len(train_loader_cifar):.4f}, Train Acc: {train_acc:.2f}%")

correct = 0
total = 0
with torch.no_grad():
    model_cifar.eval()
    for images, labels in test_loader_cifar:
        images, labels = images.to(device), labels.to(device)
        outputs = model_cifar(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"CIFAR10 Test Accuracy: {100 * correct / total:.2f}%")


Using device: cuda
Input: tensor([[[[1., 2., 3.],
          [4., 5., 6.],
          [7., 8., 9.]]]])
kernel: tensor([[[[ 1.,  0.],
          [ 0., -1.]]]])
output: tensor([[[[-4., -4.],
          [-4., -4.]]]])
SimpleCNN(
  (conv1): Conv2d(1, 8, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)
MNIST Epoch 1/10, Loss: 0.3631, Train Acc: 88.26%
MNIST Epoch 2/10, Loss: 0.0997, Train Acc: 97.00%
MNIST Epoch 3/10, Loss: 0.0735, Train Acc: 97.76%
MNIST Epoch 4/10, Loss: 0.0597, Train Acc: 98.16%
MNIST Epoch 5/10, Loss: 0.0513, Train Acc: 98.39%
MNIST Epoch 6/10, Loss: 0.0458, Train Acc: 98.62%
MNIST Epoch 7/10, Loss: 0.0421, Train Acc: 98.75%
MNIST Epoch 8/10, Loss: 